In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
print(sc)
print(sc.version)
print(sc.pythonVer)
print(sc.master)
print(str(sc.sparkHome))
print(str(sc.sparkUser()))

<SparkContext master=local[*] appName=pyspark-shell>
2.4.6
3.7
local[*]
None
joseangelvelasco


In [3]:
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession
# Create my_spark
my_spark = SparkSession.builder.getOrCreate()
# Print my_spark
print(my_spark)

In [5]:
from datetime import datetime, timedelta
import holidays
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/joseangelvelasco/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [6]:
data_path = '/Users/joseangelvelasco/GoogleDrive/DataScience/python/Data Science/dataanalytics.predictive/data/anual_data_2018.csv'

In [7]:
rdd = sc.wholeTextFiles(data_path)

In [8]:
data = sqlContext.read.format("com.databricks.spark.csv").options(header = True, inferSchema = True, sep=';', line_terminator='\n').load(path=data_path)

In [13]:


data.printSchema()

root
 |-- ID_SEGMENT: integer (nullable = true)
 |-- COD_LABORALIDAD: integer (nullable = true)
 |-- FECHA: timestamp (nullable = true)
 |-- MES: integer (nullable = true)
 |-- HORA: integer (nullable = true)
 |-- MINUTO: integer (nullable = true)
 |-- CANTIDAD_PREC: integer (nullable = true)
 |-- TOTAL_VEHICULOS: double (nullable = true)
 |-- TOTAL_VEHICULOS_1: double (nullable = true)
 |-- TOTAL_VEHICULOS_2: double (nullable = true)
 |-- TOTAL_VEHICULOS_3: double (nullable = true)
 |-- TOTAL_VEHICULOS_4: double (nullable = true)
 |-- TOTAL_VEHICULOS_5: double (nullable = true)
 |-- TOTAL_VEHICULOS_6: double (nullable = true)
 |-- TOTAL_VEHICULOS_7: double (nullable = true)
 |-- TOTAL_VEHICULOS_8: double (nullable = true)
 |-- TOTAL_VEHICULOS_9: double (nullable = true)
 |-- TOTAL_VEHICULOS_10: double (nullable = true)
 |-- TOTAL_VEHICULOS_11: double (nullable = true)
 |-- TOTAL_VEHICULOS_12: double (nullable = true)
 |-- TOTAL_VEHICULOS_13: double (nullable = true)
 |-- TOTAL_VEHICUL

In [14]:
data.collect()

[Row(ID_SEGMENT=1, COD_LABORALIDAD=4, FECHA=datetime.datetime(2018, 1, 1, 0, 0), MES=1, HORA=0, MINUTO=0, CANTIDAD_PREC=0, TOTAL_VEHICULOS=46.0, TOTAL_VEHICULOS_1=44.0, TOTAL_VEHICULOS_2=9.0, TOTAL_VEHICULOS_3=7.0, TOTAL_VEHICULOS_4=2.0, TOTAL_VEHICULOS_5=11.0, TOTAL_VEHICULOS_6=24.0, TOTAL_VEHICULOS_7=8.0, TOTAL_VEHICULOS_8=7.0, TOTAL_VEHICULOS_9=15.0, TOTAL_VEHICULOS_10=21.0, TOTAL_VEHICULOS_11=17.0, TOTAL_VEHICULOS_12=34.0, TOTAL_VEHICULOS_13=36.0, TOTAL_VEHICULOS_14=33.0, TOTAL_VEHICULOS_15=48.0, TOTAL_VEHICULOS_16=48.0, TOTAL_VEHICULOS_17=53.0, TOTAL_VEHICULOS_18=31.0, TOTAL_VEHICULOS_19=42.0, TOTAL_VEHICULOS_20=45.0, TOTAL_VEHICULOS_21=59.0, TOTAL_VEHICULOS_22=68.0, TOTAL_VEHICULOS_23=86.0, TOTAL_VEHICULOS_24=75.0),
 Row(ID_SEGMENT=2, COD_LABORALIDAD=4, FECHA=datetime.datetime(2018, 1, 1, 0, 0), MES=1, HORA=0, MINUTO=0, CANTIDAD_PREC=0, TOTAL_VEHICULOS=49.0, TOTAL_VEHICULOS_1=11.0, TOTAL_VEHICULOS_2=9.0, TOTAL_VEHICULOS_3=3.0, TOTAL_VEHICULOS_4=18.0, TOTAL_VEHICULOS_5=22.0, TOTAL

In [20]:
data.count()

1390512

The reason to use the registerTempTable( tableName ) method for a DataFrame, is so that in addition to being able to use the Spark-provided methods of a DataFrame, you can also issue SQL queries via the sqlContext.sql( sqlQuery ) method, that use that DataFrame as an SQL table. The tableName parameter specifies the table name to use for that DataFrame in the SQL queries.

Whether to use SQL or DataFrame methods like select and groupBy is probably largely a matter of preference. My understanding is that the SQL queries get translated into Spark execution plans.

In my case, I found that certain kinds of aggregation and windowing queries that I needed, like computing a running balance per customer, were available in the Hive SQL query language, that I suspect would have been very difficult to do in Spark.

If you want to use SQL, then you most likely will want to create a HiveContext instead of a regular SQLContext. The Hive query language supports a broader range of SQL than available via a plain SQLContext.

It's convenient to load the dataframe into a temp view in a notebook for example, where you can run exploratory queries on the data:

In [33]:
data.createOrReplaceTempView("data_view")

Then in another notebook you can run a sql query and get all the nice integration features that come out of the box e.g. table and graph visualisation etc.

In [49]:
%sql
SELECT * FROM data

SyntaxError: invalid syntax (<ipython-input-49-18c318626917>, line 2)

In [21]:
data.registerTempTable("data")

In [31]:
# Print the tables in the catalog
print(my_spark.catalog.listTables())

[Table(name='data', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [ ]:
df = sqlContext.table("data")
df.select('FECHA').show(10)

In [ ]:
rdd1 = df.rdd

### Rename columns

In [ ]:
df = df.withColumn("Month", month('FECHA')).withColumn("Weekday", dayofweek('FECHA')).withColumn("Hour", hour("FECHA")).withColumn("Minute", minute("FECHA"))
df.head(5)

In [ ]:
df_grouped = df.groupBy("ID_SEGMENT","Month","Weekday","Hour", "Minute").avg('TOTAL_VEHICULOS')
df_grouped.head(10)

In [ ]:
df_grouped_pandas = df_grouped.toPandas()

In [ ]:
df_grouped_pandas.head()